In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D, Activation

In [8]:
data = pd.read_csv(r'C:\Users\My PC\Documents\TXT\train.csv')
df = data

In [9]:
df[['title', 'author']] = df[['title', 'author']].fillna(value = 'Missing Value')
df = df.dropna()
df['length'] = df.iloc[:,3].str.len()
#print(df.isna().sum())
df[df['length'] < 50].count()
df = df.drop(df['text'][df['length'] < 50].index, axis=0)
df_reverse = pd.DataFrame()


#Categorical to Numeric
for col_name in df.columns:
    if(df[col_name].dtype == 'object'):
        df[col_name]= df[col_name].astype('category')
        d = dict(enumerate(df[col_name].cat.categories))
        df[col_name] = df[col_name].cat.codes
        df_reverse[col_name+"_code"] = df[col_name]
        df_reverse[col_name] = df[col_name].map(d)

features_cols = ['id', 'title', 'author', 'text']
#FEATURES AND LABELS
X = df[features_cols]
Y = df.label

C:\Users\My PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [11]:
max_features = 20800
batch_size = 32
embedding_dims = 120
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10

In [12]:
model = Sequential()

#Embedding Input Layer
model.add(Embedding(input_dim = max_features, output_dim = embedding_dims))
model.add(Dropout(0.2))

#Convolution Layer
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))

#Max Pooling Layer
model.add(GlobalMaxPooling1D())

#Vanilla Hidden Layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

#Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

W0205 17:21:56.234436 18648 deprecation.py:506] From C:\Users\My PC\Anaconda3\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0205 17:21:56.921004 18648 deprecation.py:506] From C:\Users\My PC\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 120)         2496000   
_________________________________________________________________
dropout (Dropout)            (None, None, 120)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         90250     
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               62750     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation (Activation)      (None, 250)               0

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

W0205 17:22:09.722054 18648 deprecation.py:323] From C:\Users\My PC\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 14387 samples, validate on 6167 samples
Epoch 1/10
14387/14387 [==============================] - 20s 1ms/sample - loss: 0.2766 - acc: 0.8653 - val_loss: 0.1668 - val_acc: 0.9348
Epoch 2/10
14387/14387 [==============================] - 19s 1ms/sample - loss: 0.0167 - acc: 0.9954 - val_loss: 0.2494 - val_acc: 0.9282
Epoch 3/10
14387/14387 [==============================] - 19s 1ms/sample - loss: 6.3976e-04 - acc: 0.9999 - val_loss: 0.2888 - val_acc: 0.9303
Epoch 4/10
14387/14387 [==============================] - 19s 1ms/sample - loss: 5.4643e-05 - acc: 1.0000 - val_loss: 0.3105 - val_acc: 0.9319
Epoch 5/10
14387/14387 [==============================] - 19s 1ms/sample - loss: 2.4537e-05 - acc: 1.0000 - val_loss: 0.3246 - val_acc: 0.9321
Epoch 6/10
14387/14387 [==============================] - 23s 2ms/sample - loss: 1.4502e-05 - acc: 1.0000 - val_loss: 0.3369 - val_acc: 0.9319
Epoch 7/10
14387/14387 [==============================] - 21s 1ms/sample - loss: 1.0190e-05 - acc: 1.

In [10]:
Y_pred = model.predict(X_test)
score = model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
accuracy = score[1]
print("Accuracy of CNN:", np.round(accuracy * 100), "%")

6167/6167 [==============================] - 1s 131us/sample - loss: 0.3440 - acc: 0.9285
Accuracy of CNN: 93.0 %


In [11]:
np.set_printoptions(suppress=True)
Y_pred = np.around(Y_pred).astype(int)
Y_pred = Y_pred.squeeze()

In [12]:
X_test_cp = X_test
#Decoding Data - ONE TIME STEP - DO NOT REPEAT
df_reverse.set_index('title_code', inplace=False)
title_dict = df_reverse.to_dict()['title']
df_reverse.set_index('author_code', inplace=False)
author_dict = df_reverse.to_dict()['author']
df_reverse.set_index('text_code', inplace=False)
text_dict = df_reverse.to_dict()['text']

X_test_cp['title'] = X_test_cp['title'].map(title_dict)
X_test_cp['author'] = X_test_cp['author'].map(author_dict)
X_test_cp['text'] = X_test_cp['text'].map(text_dict)
X_test_cp.set_index('id', inplace=True)

C:\Users\My PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\My PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\My PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [13]:
X_test_cp['Prediction'] = Y_pred
X_test_cp['Prediction'].replace([0,1],['Fake News','Relaible News'],inplace=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

C:\Users\My PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\My PC\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [14]:
X_test_cp.tail()

,title,author,text,Prediction
id,,,,
12652,Commissioner Starts to Press Cleveland Indians...,The New York Times,"At the Golden Globes afterparties, hoi polloi ...",Fake News
13401,Cosmology/ Love & the Afterlife,noreply@blogger.com (Alexander Light),Trending Articles: Trending Articles: ‘Chairma...,Fake News
5882,Fordham Basketball Coach Recruits With an Acad...,Dr. Patrick Slattery,Hollywood’s biggest stars took to social media...,Fake News
11500,"Carrie Fisher, Russia, Shinzo Abe: Your Wednes...",TFPP Writer,This post was originally published on this sit...,Fake News
14754,Ken Blackwell: Media ‘Borking’ Sessions Becaus...,Mac Slavo,"IQUITOS, Peru — Venezuela took its stronges...",Relaible News
